In [1]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk("C:\\mg"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
batch_size=64
epochs=100
latent_dim=256 
num_samples=10000
data_path="fra.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines) - 1)]:
    if '\t' in line:
        input_text, target_text = line.split('\t')[:2]  
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)


In [1]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
if input_texts and target_texts:
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])
else:
    max_encoder_seq_length = 0
    max_decoder_seq_length = 0

NameError: name 'input_characters' is not defined

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 57


In [6]:
input_token_index=dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i, char in enumerate(target_characters)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [10]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.1)  # Change validation_split to 0.1 or another appropriate value


Epoch 1/100
141/141 [==============================] - 29s 182ms/step - loss: 1.1514 - accuracy: 0.7400 - val_loss: 1.0413 - val_accuracy: 0.7160
Epoch 2/100
141/141 [==============================] - 24s 169ms/step - loss: 0.9184 - accuracy: 0.7512 - val_loss: 0.9706 - val_accuracy: 0.7265
Epoch 3/100
141/141 [==============================] - 24s 172ms/step - loss: 0.8242 - accuracy: 0.7706 - val_loss: 0.8437 - val_accuracy: 0.7545
Epoch 4/100
141/141 [==============================] - 24s 170ms/step - loss: 0.7198 - accuracy: 0.7937 - val_loss: 0.7663 - val_accuracy: 0.7763
Epoch 5/100
141/141 [==============================] - 24s 173ms/step - loss: 0.6604 - accuracy: 0.8082 - val_loss: 0.7173 - val_accuracy: 0.7942
Epoch 6/100
141/141 [==============================] - 24s 173ms/step - loss: 0.6224 - accuracy: 0.8180 - val_loss: 0.6686 - val_accuracy: 0.8099
Epoch 7/100
141/141 [==============================] - 24s 169ms/step - loss: 0.5943 - accuracy: 0.8265 - val_loss: 0.6619 -

In [11]:
model.save('eng2french.h5')

E:\Anaconda\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['t']] = 1.
    stop_condition = False
    decoded_sentence = 'Fire'
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence

In [19]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Go.
Decoded sentence: Firerouve !                                               
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Firea pes !                                               
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Firea pes !                                               
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run!
Decoded sentence: Fireuiessez-nous !                                        
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run!
Decoded sentence: Fireuiessez-nous !                                        
1/1 [==============================] - 0s 30ms/step
-
Input sentence: Who?
Decoded sentence: Fireuis-je ?                                              
1/1 [==============================] - 0s 30ms/step
-
Input sentence: Wow!
Decoded sentence